In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

import pandas as pd
from selenium.webdriver.chrome.service import Service

driver = webdriver.Chrome(executable_path='/Users/yoon-wonseo/pythonproject/exe/chromedriver')
driver.set_window_size(1000, 1000)

from selenium.webdriver.common.by import By
from datetime import datetime

/var/folders/xd/jtgrpp0901q9xlm5h0lqrwkc0000gn/T/ipykernel_49954/3568215962.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/yoon-wonseo/pythonproject/exe/chromedriver')


In [2]:
# 로그인 자동(이거 우회해서 보안문자 피하는 방법 있던데 시간이 없어서 못하겄다)
driver.get('https://kream.co.kr/login')



# 네이버로 로그인 클릭
driver.find_element(By.CSS_SELECTOR, 'button.btn.btn_login_naver.full.outline').click()
time.sleep(0.5)

# 새로운 탭 전환 
driver.switch_to.window(driver.window_handles[-1])


# id, pwd 입력 
id_ = input('크림 ID를 입력해주세요 : ')
pwd_ = input('크림 비밀번호를 입력해주세요 : ')


naver_id = id_
naver_pwd = pwd_

# 아이디 넣기 
id_input = driver.find_element(By.CSS_SELECTOR, '#id')

id_input.clear()
time.sleep(0.5)
id_input.send_keys(id_)

pwd_input = driver.find_element(By.CSS_SELECTOR, '#pw')
pwd_input.clear()
time.sleep(0.5)
pwd_input.send_keys(pwd_)

# 클릭 
driver.find_element(By.CSS_SELECTOR, '#log\.login').click()


크림 ID를 입력해주세요 : wonse1111
크림 비밀번호를 입력해주세요 : dnjstjWkd01


In [3]:
# 만약 비밀번호가 일치하지 않을 수도 있으니 로그인을 완료하고 이걸 눌러주자

time.sleep(0.5)

# 다시 원래 탭으로 전환 

driver.switch_to.window(driver.window_handles[0])
print('로그인을 완료했습니다.')
print('####################')

로그인을 완료했습니다.
####################


In [4]:
# 크롤링 함수 
def Kream_Sneakers_Crawling():
    
    # 3.14.1 버전은 이렇게.. 
    driver = webdriver.Chrome(executable_path='/Users/yoon-wonseo/pythonproject/exe/chromedriver')
    driver.set_window_size(1000, 1000)
    
    # 전체 신발 데이터 리스트 
    total_sneakers_data = []

    ''' 원하는 페이지 지정(최종으로 데이터 수집을 할때는 99999로 지정해야할 듯, 
                      근데 이건 다시 확인 필요함. 몇 페이지까지 신발이 있는지 범위를 지정해줘야함,
                      다섯 자리까지 같은데 확실하지 않으니 확인 필요, 그리고 신발뿐만 아니라,
                      다른 옷이나 악세사리도 나타나는 문제가 발생할 것임)
                      아니면 크림 사이트 -> shop -> 신발 카테고리에서 자동으로 스크롤 내리면서
                      신발 상품번호만 일단 긁어온 다음에, 신발 상품번호 리스트를 만들어서 상품번호를 
                      기준으로 URL을 이동하면 신발 데이터만 긁어올 수 있을듯 
    '''
    page_num = int(input('몇 페이지까지 웹스크래핑을 진행하시겠습니까? : '))

    print('##################')
    print('크림 신발 정보 웹스크래핑을 시작합니다.')

    for i in range(page_num+1):
 
        driver.get('https://kream.co.kr/products/{}'.format(i))
        # 너무 웹 사이트가 빠르게 이동하면 오류가 쉽게 나고 중간에 빼먹고 안긁어오는 값이 있으니 sleep으로 잠깐 쉬는 타임을 준다.
        # 1.5초 동안 휴식(사용자 지정: 보통 0.5) 값이 잘 안들어간다면 중간중간에 time.sleep을 넣어주자 원서상
        time.sleep(0.5)
           
        try:
            # 브랜드명
            product_brand_name = driver.find_element(By.CSS_SELECTOR,'div.detail_main_title > div > div > a').text
        except:
            continue
        
        try:
            # 상품명(영어이름)
            product_eng_name = driver.find_element(By.CSS_SELECTOR,'div.detail_main_title > div > p.title').text
        except:
            product_eng_name = ''

        try:
            # 상품명(한국이름)
            product_kor_name  = driver.find_element(By.CSS_SELECTOR, 'div.detail_main_title > div > p.sub_title').text
        except:
            product_kor_name = ''
        # 상품 정보 (해당 열부터는 결측치가 존재할 수도 있음)
        try:
            # 모델 번호 
            product_model_code = driver.find_element(By.CSS_SELECTOR,'div.detail_box.model_num > dd').text
        except:
             # 오류가 난 경우 값을 ''으로 표현(결측값)
            product_model_code = ''
        try:
            # 출시일
            product_release_date = driver.find_element(By.CSS_SELECTOR,'div.content > div > div:nth-child(2) > div > div.product_info_wrap > div > dl > div:nth-child(2) > dd').text
        except:
            product_release_date = ''

        try:
            # 발매가
            product_release_price = driver.find_element(By.CSS_SELECTOR,'div.product_info_wrap > div > dl > div:nth-child(4) > dd').text
        except:
            product_release_price = ''

        try:
            # 즉시 구매가
            product_purchase_price = driver.find_element(By.CSS_SELECTOR, 'div.container.detail > div.content > div > div:nth-child(2) > div > div.column_top > div.btn_wrap > div > a.btn_division.buy > div > span.amount > em').text
        except:
            product_purchase_price = ''

        try:
            # 즉시 판매가
            product_sale_price = driver.find_element(By.CSS_SELECTOR, 'div.container.detail > div.content > div > div:nth-child(2) > div > div.column_top > div.btn_wrap > div > a.btn_division.sell > div > span.amount > em').text
        except:
            product_sale_price = ''

        try:
            # 컬러
            product_color = driver.find_element(By.CSS_SELECTOR,'div.product_info_wrap > div > dl > div:nth-child(3) > dd').text
        except:
            product_color = ''
            
        try:
            # 최근 거래가
            product_recent_price = driver.find_element(By.CSS_SELECTOR, 'div.container.detail > div.content > div > div:nth-child(2) > div > div.column_top > div.product_figure_wrap > div.detail_price > div.price > div.amount > span.num').text
        except:
            product_recent_price = ''

        try:
            # 전 대비 증감분(증가할때랑 감소할때랑 클래스명이 달라서 중첩 try문 사용)
            try:
                product_variance = driver.find_element(By.CSS_SELECTOR, '#__layout > div > div.container.detail > div.content > div > div:nth-child(2) > div > div.column_top > div.product_figure_wrap > div.detail_price > div.price > div.fluctuation.decrease > p').text  
            except:
                product_variance = driver.find_element(By.CSS_SELECTOR, '#__layout > div > div.container.detail > div.content > div > div:nth-child(2) > div > div.column_top > div.product_figure_wrap > div.detail_price > div.price > div.fluctuation.increase > p').text  
                
        except:
            product_variance = ''

        try:
            # 가장 최근 거래일
            product_recent_date = driver.find_element(By.CSS_SELECTOR,'#panel1 > div > table > tbody > tr:nth-child(1) > td:nth-child(3)').text
        except:
            product_recent_date = ''

        try:
            # 최근 거래 사이즈
            product_recent_size = driver.find_element(By.CSS_SELECTOR, '#panel1 > div > table > tbody > tr:nth-child(1) > td:nth-child(1)').text
        except:
            product_recent_size = ''
        

        # 데이터표를 만들기 위해 저장한 값들을 리스트에 한 행씩 저장 
        total_sneakers_data.append([product_brand_name, product_eng_name, product_kor_name, product_model_code, product_release_date, product_release_price, product_purchase_price,product_sale_price,product_recent_date, product_recent_price, product_recent_size, product_variance])

    # 반복문이 끝나면 데이터프레임 생성
    data = pd.DataFrame(data=total_sneakers_data, columns=['브랜드명', '상품명(eng)', '상품명(kor)', '모델번호', '발매일', '발매가','즉시 구매가', '즉시 판매가','최근 거래일', '최근 거래가', '최근 거래 사이즈', '가격 증감']) 
    print('###########################')
    print('{}페이지까지 수집 완료'.format(i))
    print('chrome driver를 종료합니다.')
    print('###########################')
    driver.quit()
    
    # 최종 data 리턴
    return data 

In [5]:
# 크롤링 시작
data = Kream_Sneakers_Crawling()

/var/folders/xd/jtgrpp0901q9xlm5h0lqrwkc0000gn/T/ipykernel_49954/3274149593.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/yoon-wonseo/pythonproject/exe/chromedriver')


몇 페이지까지 웹스크래핑을 진행하시겠습니까? : 30
##################
크림 신발 정보 웹스크래핑을 시작합니다.
###########################
30페이지까지 수집 완료
chrome driver를 종료합니다.
###########################


In [9]:
# 현재 날짜 기준으로 저장
today_date = datetime.today().strftime("%Y_%m_%d%") 

# 데이터를 원하는 형식으로 저장(csv일 경우 to_csv)
data.to_excel('/Users/yoon-wonseo/pythonproject/크롤링/Kream_dataset_{}.xlsx'.format(today_date))

# 폴더 안을 확인해보자 